In [ ]:
# 進階open
f = open("a.txt", "w", encoding="utf-8")
f.write("abc")
f.close()
# 用with的話會自動關閉，後面要用AS來命名，且後續操作要寫在縮排中
with open("a.txt", "w", encoding="utf-8") as f:
    f.write("abc")

In [ ]:
# F12找到danmu
# url = "https://api.gamer.com.tw/anime/v1/danmu.php?videoSn=36632&geo=TW%2CHK&limit=50"
# 刪掉&含後面的LIMIT
url = "https://api.gamer.com.tw/anime/v1/danmu.php?videoSn=36632&geo=TW%2CHK"

同類型的用LIST，不同類型的用字典  
JSON格式:[ ]+{ }(網路標準格式)  
利用json.loads(str)做格式轉換

In [ ]:
# 只有from import 可以更深入, 你可以只import你要用的功能
# 不要使用 import * 會不好debug
import json
from urllib.request import urlopen

url = "https://api.gamer.com.tw/anime/v1/danmu.php?videoSn=36632&geo=TW%2CHK"
f = urlopen(url)
content = f.read()
# 有編碼的才叫字串，沒編碼的叫bytes(0101)
# print(type(content))
# print(content)

# 把bytes型態轉成字串或字典
content_conv = json.loads(content)
# print(type(content_conv))
# print(content_conv)

# content_conv: dict
# 去巴哈確認data: dict
data = content_conv["data"]
# 去巴哈確認danmus: list
danmus = data["danmu"]
# 去巴哈確認d: dict
for d in danmus:
    uid = d["userid"]
    text = d["text"]
    print(uid, text)

In [ ]:
import json
from urllib.request import urlopen


url = "https://api.gamer.com.tw/anime/v1/danmu.php?videoSn=36632&geo=TW%2CHK"
f = urlopen(url)
content = f.read()

content_conv = json.loads(content)
data = content_conv["data"]
danmus = data["danmu"]
# 不要存EXCEL，有人沒灌，有人不能解碼
# 存CSV:注意逗號，換行，雙引號
f = open("baha.csv", "w", encoding="utf-8")
f.write("uid, text\n")
for d in danmus:
    uid = d["userid"]
    text = d["text"]
    print(uid, text)
    # (always雙引號和單引號避免有彈幕含符號)
    line = '"' + uid + '", "' + text + '"\n'
    f.write(line)
f.close()

In [ ]:
# pandas函式庫(習慣簡稱pd)
import json
import pandas as pd
from urllib.request import urlopen

url = "https://api.gamer.com.tw/anime/v1/danmu.php?videoSn=36632&geo=TW%2CHK"
f = urlopen(url)
content = f.read()

content_conv = json.loads(content)
data = content_conv["data"]
danmus = data["danmu"]

# pd.DataFrame: 表格型態
df = pd.DataFrame(danmus)
# 如果你不想存列標籤(index=False)
df.to_csv("baha2.csv", encoding="utf-8")
# 唯一不用encoding="utf-8"的時候，因為excel不是這個編碼
df.to_excel("baha.xlsx")
# 不用print預覽比較漂亮
df

In [ ]:
# 後續從github讀上面取得的資料就好，不用重複做
import urllib.request as req
import pandas as pd

url = "https://github.com/X4651/tibame/raw/refs/heads/main/baha2.csv"
req.urlretrieve(url, "baha.csv")
# 如果你有存列標籤: index_col
df = pd.read_csv("baha.csv", encoding="utf-8", index_col=0)
df

HTTPError: HTTP Error 404: Not Found

In [ ]:
# pd取行: 字典
df["text"]
# 取多行:
cols = ["userid", "text"]
df[cols]

NameError: name 'df' is not defined

In [ ]:
# 過濾filter
# 過濾操作: 帶入([])跟你資料筆數依樣多的True/False
# True: 留下 False: 刪除
fil = [True, False, True]
# 取前三然後過濾
df.head(3)[fil]

In [ ]:
# 取每個人的留言次數
df["userid"].value_counts()

In [ ]:
# 找特定人的留言
q_id = "annana566"

# 給所有人一個True/False(過濾)
# df["userid"] == q_id

fil = df["userid"] == q_id
df[fil]

In [ ]:
# 想知道哪段最多彈幕
import matplotlib.pyplot as plt
# seaborn通常縮寫成sns
import seaborn as sns
# 切十段
c = pd.cut(df["time"], bins=10)
# X軸會做出直條圖(連續性)，因為另一個座標是數量，所以不用給Y
sns.countplot(x = c)
# 選轉X軸，避免擠在一起，增加可讀性
plt.xticks(rotation=90)

In [ ]:
# 想知道時間位於450~550區段的彈幕
# (and:&) (or:|)
fil = (df["time"] >= 450) & (df["time"] <= 540)
df[fil]

Request URL  
GET 直接放問號後面  
POST 秘密小紙條 藏在payload酬載

In [ ]:
# 高鐵時刻表與票價查詢
# 把明天早上九點以後的存下來
# 酬載中的資料手動轉成字典
import json
import urllib.request as request
import urllib.parse as parse

payload = {
    "SearchType":"S",
    "Lang":"TW",
    "StartStation":"TaiPei",
    "EndStation":"ZuoYing",
    "OutWardSearchDate":"2025/06/27",
    "OutWardSearchTime":"09:00",
    "ReturnSearchDate":"2025/06/26",
    "ReturnSearchTime":"14:00",
    "DiscountType":"",
}
url = "https://www.thsrc.com.tw/TimeTable/Search"

data = parse.urlencode(payload).encode()
req =  request.Request(url, data=data) # this will make the method "POST"
resp = request.urlopen(req)
content = resp.read()
# print(content)
# pandas: 表格轉換(CSV)
# 剛剛的第一種方法: list+dict一層一層去拿 幫我印出 Departure time還有Duration

# 取資料DepartureTim Duration
content_converted = json.loads(content)
data = content_converted["data"]
depart_items = data["DepartureTable"]["TrainItem"]
for d in depart_items:
    depart_time = d["DepartureTime"]
    duration = d["Duration"]
    print(depart_time, duration)

In [ ]:
# 高鐵時刻表與票價查詢
# 把明天早上九點以後的存下來
# 酬載中的資料手動轉成字典
import json
import urllib.request as request
import urllib.parse as parse
import pandas as pd

payload = {
    "SearchType":"S",
    "Lang":"TW",
    "StartStation":"TaiPei",
    "EndStation":"ZuoYing",
    "OutWardSearchDate":"2025/06/27",
    "OutWardSearchTime":"09:00",
    "ReturnSearchDate":"2025/06/26",
    "ReturnSearchTime":"14:00",
    "DiscountType":"",
}
url = "https://www.thsrc.com.tw/TimeTable/Search"

data = parse.urlencode(payload).encode()
req =  request.Request(url, data=data) # this will make the method "POST"
resp = request.urlopen(req)
content = resp.read()
# print(content)
# pandas: 表格轉換(CSV)
# 剛剛的第一種方法: list+dict一層一層去拿 幫我印出 Departure time還有Duration

# 存檔
df = pd.DataFrame(depart_items)
df.to_csv("TaiPei_ZuoYing_0627.csv", encoding="utf-8")

In [ ]:
a = 5
b = 6
s = str(a) + "-" + str(b)
# s = f"{a}-{b}"
pat = "{}-{}"
s = pat.format(a, b)
s

In [ ]:
import json
import urllib.request as request
import urllib.parse as parse
import pandas as pd

start_station = "TaiPei"
end_station = "ZuoYing"
depart_date = "2025/06/27"
payload = {
    "SearchType":"S",
    "Lang":"TW",
    "StartStation":start_station,
    "EndStation":end_station,
    "OutWardSearchDate":depart_date,
    "OutWardSearchTime":"09:00",
    "ReturnSearchDate":"2025/06/26",
    "ReturnSearchTime":"14:00",
    "DiscountType":"",
}
url = "https://www.thsrc.com.tw/TimeTable/Search"

data = parse.urlencode(payload).encode()
req =  request.Request(url, data=data) # this will make the method "POST"
resp = request.urlopen(req)
content = resp.read()
# print(content)
# pandas: 表格轉換(CSV)
# 剛剛的第一種方法: list+dict一層一層去拿 幫我印出 Departure time還有Duration
content_converted = json.loads(content)
data = content_converted["data"]
depart_items = data["DepartureTable"]["TrainItem"]


df = pd.DataFrame(depart_items)
fn = "{}_{}_{}.csv".format(start_station,
                           end_station,
                           "20250627")
df.to_csv(fn, encoding="utf-8")

In [ ]:
df

In [ ]:
from datetime import date
from datetime import timedelta
delta = timedelta(days=5)
newdate = date.today() + delta
# strftime: time->str
# strptime: str->time
pat = "%Y/%m/%d"
newdate.strftime(pat)

In [ ]:
def add(n1, n2):
    ans = n1 + n2
    return ans
add(3, 5)

8

In [ ]:
# 下載接下來十天的資料
from datetime import date
from datetime import timedelta

def save_timetable(date):
    start_station = "TaiPei"
    end_station = "ZuoYing"
    # datetime型態
    depart_date = date.strftime("%Y/%m/%d")
    payload = {
        "SearchType":"S",
        "Lang":"TW",
        "StartStation":start_station,
        "EndStation":end_station,
        "OutWardSearchDate":depart_date,
        "OutWardSearchTime":"09:00",
        "ReturnSearchDate":"2025/06/26",
        "ReturnSearchTime":"14:00",
        "DiscountType":"",
    }
    url = "https://www.thsrc.com.tw/TimeTable/Search"

    data = parse.urlencode(payload).encode()
    req =  request.Request(url, data=data) # this will make the method "POST"
    resp = request.urlopen(req)
    content = resp.read()
    # print(content)
    # pandas: 表格轉換(CSV)
    # 剛剛的第一種方法: list+dict一層一層去拿 幫我印出 Departure time還有Duration
    content_converted = json.loads(content)
    data = content_converted["data"]
    depart_items = data["DepartureTable"]["TrainItem"]


    df = pd.DataFrame(depart_items)
    fn = "{}_{}_{}.csv".format(start_station,
                            end_station,
                            date.strftime("%Y%m%d"))
    df.to_csv(fn, encoding="utf-8")

# 測試函式功能
# delta = timedelta(days=5)
# newdate = date.today() + delta
# save_timetable(newdate)

today = date.today()
for i in range(10):
    delta = timedelta(days=i+1)
    target_date = today + delta
    # 查看下載進度
    print(f"正在下載 {target_date}")
    save_timetable(target_date)
print(f"{i+1}筆資料下載完畢")

正在下載 2025-06-27
正在下載 2025-06-28
正在下載 2025-06-29
正在下載 2025-06-30
正在下載 2025-07-01
正在下載 2025-07-02
正在下載 2025-07-03
正在下載 2025-07-04
正在下載 2025-07-05
正在下載 2025-07-06
10筆資料下載完畢


In [ ]:
# 改成讓使用者輸入 不可能讓他用輸入 讓他選數字就好 降低輸入錯的機率
from datetime import date
from datetime import timedelta

def save_timetable(start_s, end_s, date):
    # start_station = "TaiPei"
    start_station = start_s
    # end_station = "ZuoYing"
    end_station = end_s
    # datetime型態
    depart_date = date.strftime("%Y/%m/%d")
    payload = {
        "SearchType":"S",
        "Lang":"TW",
        "StartStation":start_station,
        "EndStation":end_station,
        "OutWardSearchDate":depart_date,
        "OutWardSearchTime":"09:00",
        "ReturnSearchDate":"2025/06/26",
        "ReturnSearchTime":"14:00",
        "DiscountType":"",
    }
    url = "https://www.thsrc.com.tw/TimeTable/Search"

    data = parse.urlencode(payload).encode()
    req =  request.Request(url, data=data) # this will make the method "POST"
    resp = request.urlopen(req)
    content = resp.read()
    # print(content)
    # pandas: 表格轉換(CSV)
    # 剛剛的第一種方法: list+dict一層一層去拿 幫我印出 Departure time還有Duration
    content_converted = json.loads(content)
    data = content_converted["data"]
    depart_items = data["DepartureTable"]["TrainItem"]


    df = pd.DataFrame(depart_items)
    fn = "{}_{}_{}.csv".format(start_station,
                            end_station,
                            date.strftime("%Y%m%d"))
    df.to_csv(fn, encoding="utf-8")

# test
# delta = timedelta(days=5)
# newdate = date.today() + delta
# save_timetable(newdate)
start = "TaiPei"
end = "ZuoYing"
today = date.today()
for i in range(2):
    delta = timedelta(days=i+1)
    target_date = today + delta
    print(f"正在下載 {target_date}")
    save_timetable(start, end, target_date)
print(f"{i+1}筆資料下載完畢")